In [1]:
import pygame
import os
import neat
import random
pygame.font.init()

WIN_WIDTH = 1200
WIN_HEIGHT = 650

win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
MOTOR_IMG = pygame.image.load(os.path.join('motor', 'motor.png')).convert_alpha()
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join('motor', 'back.png'))).convert_alpha()
STONE_IMG = pygame.image.load(os.path.join('motor', 'stone.png')).convert_alpha()

STAT_FONT = pygame.font.SysFont("comicsans", 50)

fps = 30
motor_origin_y = 320
GEN = 0

# Difficulty
pace = 40
jump_height = 200
gravity = 35

# NeuralNet
punishment = 5
reward = 5
per_frame_reward = 0.0 #0.1
jumping_thresh = 0.9 # only 80% confidence minimum allowed for action


class Motor:
    IMG = MOTOR_IMG
    going_up = True
    jump_height = jump_height
    origin_y = motor_origin_y

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.going_up = self.going_up
        self.jumping = False

    def jump(self):
        if self.y == self.origin_y:
            self.going_up = True

        if self.going_up:
            self.y -= gravity
            if self.y <= (self.origin_y - self.jump_height):
                self.going_up = False
        else:
            self.y += gravity
            if self.y >= self.origin_y:
                self.jumping = False

    def draw(self, win):
        win.blit(self.IMG, (self.x, self.y))

    def get_mask(self):
        return pygame.mask.from_surface(self.IMG)

class Stone:
    VEL = pace
    IMG = STONE_IMG
    appearing_y_level = motor_origin_y + 150
    WINDOW_WIDTH = WIN_WIDTH

    def __init__(self):
        self.appearing_x_level = self.WINDOW_WIDTH + random.randrange(0, 800)
        self.passed = False

    def move(self):
        self.appearing_x_level -= self.VEL

    def draw(self, win):
        win.blit(self.IMG, (self.appearing_x_level, self.appearing_y_level))

    def collide(self, motor):
        motor_mask = motor.get_mask()
        stone_mask = pygame.mask.from_surface(self.IMG)

        # calculate offset between masks
        offset = (self.appearing_x_level - motor.x, self.appearing_y_level - motor.y)

        collided = motor_mask.overlap(stone_mask, offset)

        if collided:
            return True
        return False

class Background:
    VEL = pace
    WIDTH = BG_IMG.get_width()
    IMG = BG_IMG

    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH

    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH

        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))


def draw_window(win, background, motors, stones, score, gen):
    background.draw(win)
    for stone in stones:
        stone.draw(win)

    for motor in motors:
        motor.draw(win)

    text = STAT_FONT.render("Score: " + str(score), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))

    text = STAT_FONT.render("Gen: " + str(gen), 1, (255, 255, 255))
    win.blit(text, (10, 10))

    pygame.display.update()

def fitness_function(genomes, config):
    global Jump
    global GEN
    global punishment
    global per_frame_reward
    global reward
    global jumping_thresh
    GEN += 1
    nets = []
    ge = []
    motors = []
    stones = [Stone()]
    clock = pygame.time.Clock()
    background = Background(-185)
    score = 0
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    background.draw(win)
    win.blit(BG_IMG, (0, 0))

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        motors.append(Motor(100, motor_origin_y))
        g.fitness = 0 #initial fitness to 0
        ge.append(g)

    run = True
    while run:
        clock.tick(fps)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()

        background.move()
        stone_index = 0
        if len(motors)>0:
            if len(stones) > 1 and motors[0].x > stones[0].appearing_x_level + stones[0].IMG.get_width(): # if we passed this one then change index to the stone after it
                stone_index = 1
        else:
            # no birds left then quit
            run = False
            break

        for x, motor in enumerate(motors):
            ge[x].fitness += per_frame_reward # this runs 30 times a second
            '''
            MODIFY CONFIG FILE AS TO ACCEPT X DISTANCE IN 2ND ARGUMENT NOT TOP GAP
            '''
            output = nets[x].activate((motor.y, abs(motor.x - stones[stone_index].appearing_x_level), abs(motor.y - stones[stone_index].appearing_y_level))) # abs = absolute value

            if output[0] > jumping_thresh: # in other systems u haVE MORE THAN ONE OUTPUT NEURON, but here we only hve one but still
                motor.jumping = True

        for motor in motors:
            if motor.jumping:
                motor.jump()

        rem = []
        add_stone = False
        for stone in stones:
            for x, motor in enumerate(motors):
                if stone.collide(motor):
                    ge[x].fitness -= punishment #encourage birds to to in between the pipes
                    #birds.remove(bird) # remove it from the screen as we don't WANT IT TO CONTINUE PLAYING
                    motors.pop(x) # issue removing birds while looping, will be fixed
                    nets.pop(x)
                    ge.pop(x)

                if not stone.passed and stone.appearing_x_level < motor.x:
                    stone.passed = True
                    add_stone = True
                    score += 1

            if stone.appearing_x_level + stone.IMG.get_width() < 0:
                rem.append(stone)

            stone.move()

        if add_stone:
            # give them a high fitness to encourage them to go in between the pipes
            for g in ge:
                g.fitness += reward
            stones.append(Stone())

        for r in rem:
            stones.remove(r)

        win.fill((0,0,0))
        draw_window(win, background, motors, stones, score, GEN)

def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    winner = p.run(fitness_function ,1000000) # how many generations

#if __name__ == "__main__":
#local_dir = os.path.dirname(__file__)
config_path = os.path.join("config.txt")
run(config_path)


pygame 2.0.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 

Population's average fitness: -3.66667 stdev: 5.46707
Best fitness: 25.00000 - size: (1, 3) - species 1 - id 17
Average adjusted fitness: 0.044
Mean genetic distance 1.115, standard deviation 0.395
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30     25.0    0.044     0
Total extinctions: 0
Generation time: 10.257 sec

 ****** Running generation 1 ****** 



error: display Surface quit